In [19]:
import os
import pandas as pd
import gmaps
import googlemaps as g
import yaml
import numpy

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.notebook_repr_html', True)

# Loading dataset
df = pd.read_csv(os.path.join('..', 'datasets', 'Restaurant_Scores_Cleaned.csv'))
df = df.set_index('business_id')

# Applying google credentials
with open('google.yaml', 'r') as f:
    google_credentials = yaml.load(f)

google_api_key = google_credentials['api-key']
gmaps.configure(api_key = google_api_key)



df

IOError: [Errno 2] No such file or directory: 'google.yaml'

In [3]:
#Dropping unnecessary address colums
df.drop(['business_location'], inplace = True, axis = 1)

print len(df)
df.head()

53729


,business_name,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,address,location
business_id,,,,,,,,,,
10,Tiramisu Kitchen,10_20140114,01/14/2014 12:00:00 AM,92.0,Routine - Unscheduled,10_20140114_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk,033 Belden Pl San Francisco,"(37.791116, -122.403816)"
10,Tiramisu Kitchen,10_20140114,01/14/2014 12:00:00 AM,92.0,Routine - Unscheduled,10_20140114_103145,Improper storage of equipment utensils or linens,Low Risk,033 Belden Pl San Francisco,"(37.791116, -122.403816)"
10,Tiramisu Kitchen,10_20140114,01/14/2014 12:00:00 AM,92.0,Routine - Unscheduled,10_20140114_103154,Unclean or degraded floors walls or ceilings,Low Risk,033 Belden Pl San Francisco,"(37.791116, -122.403816)"
10,Tiramisu Kitchen,10_20140729,07/29/2014 12:00:00 AM,94.0,Routine - Unscheduled,10_20140729_103144,Unapproved or unmaintained equipment or utensils,Low Risk,033 Belden Pl San Francisco,"(37.791116, -122.403816)"
10,Tiramisu Kitchen,10_20140729,07/29/2014 12:00:00 AM,94.0,Routine - Unscheduled,10_20140729_103129,Insufficient hot water or running water,Moderate Risk,033 Belden Pl San Francisco,"(37.791116, -122.403816)"


In [4]:
# Identifying Nulls 
null_data = df[df['location'].isnull()]
null_data.head()
len(null_data)



93

In [5]:
# Drop all nulls since is a small number
df.dropna(subset = ['location'], inplace = True)
len(df)

53636

In [6]:
# Dropping rows where lat long = 0,0
df = df[df.location != '(0, 0)']
len(df)

53567

In [7]:
df['inspection_score'].value_counts()

90.0     3671
92.0     3367
94.0     3205
96.0     2729
100.0    2716
88.0     2305
86.0     2161
83.0     1379
84.0     1328
85.0     1325
         ... 
59.0       68
61.0       66
56.0       43
54.0       41
58.0       35
53.0       24
47.0       14
46.0       13
52.0       11
50.0       11
Name: inspection_score, Length: 49, dtype: int64

In [8]:
df['inspection_type'].value_counts()

Routine - Unscheduled                41175
Reinspection/Followup                 5502
New Ownership                         1915
Complaint                             1892
New Construction                      1685
Non-inspection site visit              864
Structural Inspection                  162
Complaint Reinspection/Followup        144
Routine - Scheduled                    131
Foodborne Illness Investigation         84
Special Event                            6
Administrative or Document Review        5
Community Health Assessment              1
Multi-agency Investigation               1
Name: inspection_type, dtype: int64

In [9]:
df['violation_description'].value_counts()

Unclean or degraded floors walls or ceilings                          3563
Moderate risk food holding temperature                                2799
Inadequate and inaccessible handwashing facilities                    2605
Unapproved or unmaintained equipment or utensils                      2424
Inadequately cleaned or sanitized food contact surfaces               2315
Wiping cloths not clean or properly stored or inadequate sanitizer    2292
Improper food storage                                                 2142
Foods not protected from contamination                                1896
Unclean nonfood contact surfaces                                      1841
High risk food holding temperature                                    1627
                                                                      ... 
Reservice of previously served foods                                     7
Unreported or unrestricted ill employee with communicable disease        6
Discharge from employee n

In [11]:
df['risk_category'].value_counts()


Low Risk         20890
Moderate Risk    14247
High Risk         5601
Name: risk_category, dtype: int64

In [12]:
# # Identifying Nulls for risk category
# null_data = df[df['risk_category'].isnull()]
# null_data.head()
# len(null_data)

In [14]:
df['risk_category'].fillna('No Risk', inplace=True)

In [15]:
df['risk_category'].value_counts()

Low Risk         20890
Moderate Risk    14247
No Risk          12829
High Risk         5601
Name: risk_category, dtype: int64

In [16]:
color = df.risk_category.map({'High Risk': 'red', 'Moderate Risk': 'orange', 'Low Risk': 'yellow' , 'No Risk': 'green'})